In this notebook we show how to prepare all the input and parameter files necessary to run Diva4D.<br>
The code rely on 2 modules:
* [diva2Dwrite](./diva2Dwrite.py), for the preparation of the input files for the 2D. 
* [diva4Dwrite](./diva2Dwrite.py), for the preparation of the input and configuarion files for the 4D. 

In [41]:
import os
import logging
import shutil
import subprocess
import diva2Dwrite
import diva4Dwrite
import importlib

In [54]:
print('f')

f


Reload the module (only during code development)

In [21]:
importlib.reload(diva4Dwrite)
importlib.reload(diva2Dwrite)

<module 'diva2Dwrite' from '/home/ctroupin/Projects/SeaDataCloud/Python/diva2Dwrite.py'>

# User inputs and configuration

Indicate the path to the Diva installation you want to work with (diva-x.y.z).

In [55]:
divabasedir = "/home/ctroupin/Software/DIVA/diva-4.7.1"

## List of user parameters

Set the different parameters necessary for a 4D analysis.<br>
For clarity, they are separated by input files.

### List of ODV spreadsheet files

In [56]:
datafilelist = ['./blacksea_data_CTD.txt']

### Parameters for the advection constraint and the reference field

* advection_flag: Flag that indicates if advection is activated
* ref_flag: Flag that indicates if there is a reference field
* var_year_code: Variable year code
* var_month_code: Variable month code

In [7]:
advection_flag, ref_flag, var_year_code, var_month_code = 0, 0, '00000000', '0000'

### Parameters for the 4D analysis

* extraction_flag: Data extraction: 1 do it, 0 do nothing, -1 press coord, -10 pressure+Saunders
* coast_flag: boundary lines and coastlines generation: 0 nothing, 1: contours, 2: UV, 3: 1+2
* clean_flag: cleaning data on mesh: 1, 2: RL, 3: both, 4: 1 + outliers elimination, 5: =4+2
* min_datanum: minimal number of data in a layer. If less, uses data from any month.
* param_flag: Parameters estimation and vertical filtering:
* min_l, max_l: minimal/maximal correlation length
* min_snr, max_snr: minimal/maximal signal-to-noise ratio
* analysisref_flag: Analysis and reference field
* lower_level, upper_level: lower/upper level numbers
* netcdf4d_flag: 4D netcdf files generation
* gnuplot_flag: gnuplot plots: 0 or 1
* detrend_groupnum: Data detrending: number of groups, 0 if no detrending.

In [8]:
extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l, max_l, min_snr, max_snr,\
analysisref_flag, lower_level, upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum =\
1, 1, 0, 0, -30, 1.4, 10., 0.1, 5.0, 1, 1, 3, 0, 0, 0

### Parameters for the netCDF file

* ncdf_title: Title string for 3D NetCDF file
* ncdf_reftime: Reference time for data (ie: days since since 1900-01-01), if not climatological data
* ncdf_timeval: Time value (that reprsents the data set), if not climatological data
* ncdf_cellmethod: Cell_methode string:
* ncdf_institution: Institution name: where the dataset was produced.
* ncdf_groupemail: Production group and e-mail
* ncdf_source: Source (observation, radiosonde, database, model-generated data,...)
* ndf_comment: Comment
* ncdf_authoremail: Autor e-mail adress (or contact person to report problems)
* ncdf_acknowlegment: Acknowledgement

In [9]:
ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
ncdf_institution, ncdf_groupemail, ncdf_source,\
ncdf_comment, ncdf_authoremail, ncdf_acknowlegment =\
"4D Test on the Black Sea", 'months since since xxxx-01-01', 1200, 'time: mean (this month data from all years)',\
'University of Liege, GeoHydrodynamics and Environment Research',\
'Diva group. E-mails : a.barth@ulg.ac.be ; swatelet@ulg.ac.be',\
'data_from various sources for diva software testing work', 'No comment', 'swatelet@ulg.ac.be',\
'No acknowledgement'

### Parameters for the output grid

* xmin, ymin: longitute and latitude of the lower-left corner of the domain
* nx, ny: number of cells in the x and y directions
* dx, dy: spatial steps in the x and y directions

In [33]:
xmin, ymin, nx, ny, dx, dy = 27.0, 40.0, 151, 76, 0.1, 0.1

### Parameters for the analysis

In [34]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -99.
iCoordChange = 2
iSpec = 11
iReg = 2

# Create the new input files

## Generate the names of the sub-directories

In [42]:
divabindir, divasrcdir, diva2Ddir, diva4Ddir, diva4Dinputdir = diva2Dwrite.make_diva_directorynames(divabasedir)
print(diva4Dinputdir)

/home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input


## Write the different input files using the functions of diva4Dwrite

In [43]:
diva4Dwrite.write_datasource(diva4Ddir, datafilelist)

In [44]:
diva4Dwrite.write_constandrefe(diva4Ddir, advection_flag, ref_flag, var_year_code, var_month_code)

In [45]:
diva4Dwrite.write_driver(diva4Ddir, extraction_flag, coast_flag, clean_flag, min_datanum, param_flag,
                         min_l, max_l, min_snr, max_snr,
                         analysisref_flag, lower_level, upper_level, 
                         netcdf4d_flag, gnuplot_flag, detrend_groupnum)

In [59]:
diva4Dwrite.write_monthlist(diva4Ddir, ['0101', '0202', '0303'])

In [60]:
diva4Dwrite.write_qflist(diva4Ddir, [0, 1])

In [61]:
diva4Dwrite.write_varlist(diva4Ddir, ['Temperature', 'Salinity'])

In [62]:
diva4Dwrite.write_yearlist(diva4Ddir, ['19002009',])

In [63]:
diva4Dwrite.write_contourdepth(diva4Dinputdir, [30, 20, 10, 0])

In [51]:
diva4Dwrite.write_ncdfinfo(diva4Dinputdir, ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,
                   ncdf_institution, ncdf_groupemail, ncdf_source,
                   ncdf_comment, ncdf_authoremail, ncdf_acknowlegment)

In [52]:
diva2Dwrite.write_parameter(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField, os.path.join(diva4Dinputdir, 'param.par'))

# Run divadoall (4D analysis)

In [53]:
divadoall_script = './divadoall'
env = os.environ.copy()
env["PATH"] = ".:" + env["PATH"]

p = subprocess.Popen(divadoall_script, stdout=subprocess.PIPE, cwd=diva4Ddir, env=env)
out = p.stdout.read()

print(out)

b"divadoall: divadoall run starts at:\nTue Jan 24 15:05:25 CET 2017\ndivadoall:  Start divadoall: idorefe = 0\ndivadoall:                   idoanal = 1\ndivadoall: Working in /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology\ndivadoall: Cleaning up the old NC info and list files\ndivadoall: variable Temperature\ndivadoall: year 19002009\ndivadoall: variable Salinity\ndivadoall: year 19002009\ndivadoall: Cleaning up the old data files\ndivadoall: variable Temperature\ndivadoall: year 19002009\ndivadoall: month 0101\ndivadoall: variable Salinity\ndivadoall: year 19002009\ndivadoall: month 0101\ndivadoall: Data source file is ./blacksea_data_CTD.txt\ndivadoall: Trying to guess ODV format on ./blacksea_data_CTD.txt\n2331776 and 145736\n11, 16\nNo depth axis (or speed+depth) ?\nno.\ndivadoall: Data source file is ./blacksea_data_CTD.txt\ndivadoall: Trying to guess ODV format on ./blacksea_data_CTD.txt\n2331776 and 145736\n11, 16\n./blacksea_data_CTD.txt\ndivadoall: Now extracting\nCr